In [1]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00


Creating dataset, merging two different json files. One contains sentences with mountains and one contains sentences with no mountains. First part was manually collected using ChatGPT and the secong obtained from kaggle-entity-annotated-corpus-ner-dataset.

In [2]:
import json
from datasets import load_dataset

In [3]:
# Files paths
first_json_file = "/content/data_200_mountains.json"
second_json_file = "/content/data_200_no_mountains.json"

In [4]:
# First file is already loaded, we should write the second file with kagle dataset samples

# Loading the dataset
dataset = load_dataset('rjac/kaggle-entity-annotated-corpus-ner-dataset')

# Select the first 200 samples
subset = dataset['train'].select(range(200))

# Add custom IDs from 201 to 400 and adjust ner_tags
subset_filtered = subset.map(lambda x, idx: {
    'id': str(201 + idx),
    'tokens': x['tokens'],
    'ner_tags': [0 if tag != 0 else tag for tag in x['ner_tags']]
}, with_indices=True)

# Remove any other fields
subset_filtered = subset_filtered.remove_columns([col for col in subset_filtered.column_names if col not in ['id', 'tokens', 'ner_tags']])

# Save as JSON
subset_filtered.to_json(second_json_file, orient='records', lines=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

(…)-00000-of-00004-e7c482bd9e74046f.parquet:   0%|          | 0.00/857k [00:00<?, ?B/s]

(…)-00001-of-00004-8e5af59a8884c020.parquet:   0%|          | 0.00/860k [00:00<?, ?B/s]

...

(…)-00003-of-00004-d4161cf902a01090.parquet:   0%|          | 0.00/823k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/47959 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

52073

In [5]:
# Loading files
def load_json(file_path):
    """
    Function for loading json file by specified file_path
    """
    with open(file_path, 'r', encoding='utf-8') as json_file:
        return json.load(json_file)

# Concat json files
def merge_json_files(file1_path, file2_path, output_path):
    # Load the contents of both JSON files
    data1 = load_json(file1_path)
    data2 = load_json(file2_path)

    # Assuming both are lists, concatenate them
    merged_data = data1 + data2

    # Write the merged data to a new JSON file
    with open(output_path, 'w', encoding='utf-8') as output_file:
        json.dump(merged_data, output_file, ensure_ascii=False, indent=4)

In [6]:
output_path = 'result_data.json'

# Merge the JSON files
merge_json_files(first_json_file, second_json_file, output_path)